In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Datasets
features = {
    'Color': ['Red', 'Green', 'Blue', 'Green', 'Red'],
    'Type': ['SUV', 'Sedan', 'SUV', 'Truck', 'Sedan'],
    'Origin': ['Domestic', 'Imported', 'Imported', 'Domestic', 'Domestic']
}

labels = {
    'Stolen': ['Yes', 'No', 'No', 'Yes', 'No']
}

# Adding in DataFrames
df_features = pd.DataFrame(features)
df_labels = pd.DataFrame(labels)

# Convert 'Stolen' labels to numerical values
label_map = {'Yes': 1, 'No': 0}
encoded_labels = np.array([label_map[label] for label in df_labels['Stolen']])

# Convert categorical features using NumPy to numerical values
def encode_features(df):
    color_map = {'Red': 0, 'Green': 1, 'Blue': 2}
    type_map = {'SUV': 0, 'Sedan': 1, 'Truck': 2}
    origin_map = {'Domestic': 0, 'Imported': 1}
    
    encoded = np.zeros((len(df), 3), dtype=int)
    encoded[:, 0] = [color_map[color] for color in df['Color']]
    encoded[:, 1] = [type_map[type_] for type_ in df['Type']]
    encoded[:, 2] = [origin_map[origin] for origin in df['Origin']]
    return encoded

encoded_features = encode_features(df_features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(encoded_features, encoded_labels, test_size=0.2, random_state=42)

# Training the models
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)

nb = GaussianNB()
nb.fit(X_train, y_train)
nb_predictions = nb.predict(X_test)

svm = SVC(kernel='linear', C=1.0, probability=True, random_state=42)  # Set probability=True for SVM
svm.fit(X_train, y_train)
svm_predictions = svm.predict(X_test)

# Create a voting classifier combining all classifiers with soft voting
voting_clf = VotingClassifier(estimators=[('knn', knn), ('nb', nb), ('svm', svm)], voting='soft')

# Train the voting classifier
voting_clf.fit(X_train, y_train)

# Predict using the voting classifier
voting_predictions = voting_clf.predict(X_test)

# Calculate agreements between classifiers
def find_agreement(predictions1, predictions2):
    return np.mean(predictions1 == predictions2)

# Predict probabilities for each class using all classifiers
knn_probabilities = knn.predict_proba(X_test)
nb_probabilities = nb.predict_proba(X_test)
svm_probabilities = svm.predict_proba(X_test)

# Find agreements between classifiers
agreements = {
    ('KNN', 'Naive Bayes'): find_agreement(knn_predictions, nb_predictions),
    ('KNN', 'SVM'): find_agreement(knn_predictions, svm_predictions),
    ('Naive Bayes', 'SVM'): find_agreement(nb_predictions, svm_predictions)
}

# Sort agreements to find which two classifiers have highest agreement
sorted_agreements = sorted(agreements.items(), key=lambda x: x[1], reverse=True)

# Print agreements
print("Agreements between classifiers:")
for (clf1, clf2), agreement in sorted_agreements:
    print(f"{clf1} and {clf2}: {agreement:.2f}")

# Final output
print("\nVoting Classifier Accuracy:", accuracy_score(y_test, voting_predictions))
print("\nVoting Classifier Classification Report:\n", classification_report(y_test, voting_predictions))

# User interaction for predictions
while True:
    color_input = input("Enter the color (Red, Green, Blue) (or type 'quit' to exit): ")
    if color_input.lower() == 'quit':
        break
    type_input = input("Enter vehicle type (SUV, Sedan, Truck) (or type 'quit' to exit): ")
    if type_input.lower() == 'quit':
        break
    origin_input = input("Enter car origin (Domestic, Imported) (or type 'quit' to exit): ")
    if origin_input.lower() == 'quit':
        break
    
    def encode_input(color, type_, origin):
        color_map = {'Red': 0, 'Green': 1, 'Blue': 2}
        type_map = {'SUV': 0, 'Sedan': 1, 'Truck': 2}
        origin_map = {'Domestic': 0, 'Imported': 1}
        return np.array([color_map[color], type_map[type_], origin_map[origin]])

    encoded_input = encode_input(color_input, type_input, origin_input)
    
    knn_prediction = knn.predict([encoded_input])
    nb_prediction = nb.predict([encoded_input])
    svm_prediction = svm.predict([encoded_input])
    voting_prediction = voting_clf.predict([encoded_input])

    print("\nKNN Prediction:", 'Yes' if knn_prediction[0] == 1 else 'No')
    print("Naive Bayes Prediction:", 'Yes' if nb_prediction[0] == 1 else 'No')
    print("SVM Prediction:", 'Yes' if svm_prediction[0] == 1 else 'No')
    print("Voting Classifier Prediction:", 'Yes' if voting_prediction[0] == 1 else 'No')


Agreements between classifiers:
KNN and Naive Bayes: 1.00
KNN and SVM: 0.00
Naive Bayes and SVM: 0.00

Voting Classifier Accuracy: 1.0

Voting Classifier Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

Enter the color (Red, Green, Blue) (or type 'quit' to exit): Blue
Enter vehicle type (SUV, Sedan, Truck) (or type 'quit' to exit): SUV
Enter car origin (Domestic, Imported) (or type 'quit' to exit): Domestic

KNN Prediction: No
Naive Bayes Prediction: Yes
SVM Prediction: No
Voting Classifier Prediction: Yes
Enter the color (Red, Green, Blue) (or type 'quit' to exit): Blue
Enter vehicle type (SUV, Sedan, Truck) (or type 'quit' to exit): SUV
Enter car origin (Domestic, Imported) (or type 'quit' to exit): Imported

KNN Prediction: No
Naive Bayes 